In [19]:
from datetime import datetime
from datetime import date
from pathlib import Path
from zoneinfo import ZoneInfo
import os
import pandas as pd
import math

from neuroconv.datainterfaces import TDTFiberPhotometryInterface
from neuroconv.utils import dict_deep_update, load_dict_from_file
from neuroconv.tools import configure_and_write_nwbfile
from pynwb.image import ImageSeries
from pynwb.epoch import TimeIntervals

import tdt

from ndx_manoli_meta import AssayMetadata,IndividualMetadata

In [22]:
# load metadata table
metatable = pd.read_csv('metadata_nwb_introduction_v2.csv')
metatable.head()

# load static photometry metadata object
# currently in a cell below, but I should move it to some kind of file

,Eartag,AssayType,RecordingDate,Sex,Genotype,DOB,Family,SurgeryDate,TDTGroupPath,TDTTankPath,...,ScoreFile,PreprocessedDataPath,PreprocessedFiledFF,PreprocessedFileTrimdFF,PartnerEartag,PartnerSex,PartnerGenotype,PartnerDOB,PartnerFamily,PairDate
0,V1259,introduction,20181205,M,WT,20181001,FC,20181115,M:\photometry-Long2025\tdt_files\1 Intros,V1259-181205-134533,...,V1259 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V1259_Intro_dFF.mat,V1259_Intro_Trim_dFF.mat,NaN,NaN,NaN,NaN,NaN,20181205
1,V2210,introduction,20190218,M,WT,20181223,B5,20190125,M:\photometry-Long2025\tdt_files\1 Intros,V2210-190218-142548,...,V2210 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V2210_Intro_dFF.mat,V2210_Intro_Trim_dFF.mat,NaN,NaN,NaN,NaN,NaN,20190218
2,V2211,introduction,20190218,M,WT,20181223,B5,20190130,M:\photometry-Long2025\tdt_files\1 Intros,V2211-190218-133221,...,V2211 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V2211_Intro_dFF.mat,V2211_Intro_Trim_dFF.mat,NaN,NaN,NaN,NaN,NaN,20190218
3,V3950,introduction,20190904,M,WT,20190706,G15,20190808,M:\photometry-Long2025\tdt_files\1 Intros,V3950-190904-140232,...,V3950 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V3950_Intro_dFF.mat,V3950_Intro_Trim_dFF.mat,V4146,F,WT,20190726.0,NG5,20190904
4,V5473,introduction,20200212,M,WT,20191213,G21,20200123,M:\photometry-Long2025\tdt_files\1 Intros,V5473-200212-132254,...,V5473 Intro.csv,D:\photometry_nwb\UCSF02 NAc Photometry\Pre pr...,V5473_Intro_dFF.mat,V5473_Intro_Trim_dFF.mat,V5552,F,WT,20191223.0,H5,20200212


In [3]:
# LOOP OVER METADATA TABLE
# just use first line for now
i=0
folder_path = os.path.join(metatable.TDTGroupPath[i],metatable.TDTTankPath[i])
# folder_path = 'D:\\photometry_nwb\\UCSF02 NAc Photometry\\Raw Data\\1 Intros\\V2210-190218-142548'

In [23]:
#### BUILD A FILE

# START TDTFiber interface
interface = TDTFiberPhotometryInterface(folder_path=folder_path, verbose=False)
metadata = interface.get_metadata()

# CHECK SESSION START TIME
sessionstart = metadata['NWBFile']['session_start_time']
# reformat
recd = str(metatable.RecordingDate[i])
sessiondate = sessionstart[0:10].replace('-','')
if sessiondate != recd:
    print(f'Session date {sessiondate} and recording date {metatable.RecordingDate[i]} mismatch for {metatable.Eartag[i]} {metatable.AssayType[i]}.')
    # break

# ADD SUBJECT METADATA

# calculate age at recording
dobstr = str(metatable.DOB[i])
dob = date(int(dobstr[0:4]),int(dobstr[4:6]),int(dobstr[6:]))
recobj = date(int(recd[0:4]),int(recd[4:6]),int(recd[6:]))
delta = recobj-dob
agei = delta.days

# calculate days post pairing
pairstr = str(metatable.PairDate[i])
paird = date(int(pairstr[0:4]),int(pairstr[4:6]),int(pairstr[6:]))
delta = recobj-paird
dpostpair = delta.days

metadata["Subject"] = dict(
    subject_id=f"{metatable.Eartag[i]}",
    sex=f"{metatable.Sex[i]}",
    age=f"P{agei}D",
    species="Microtus ochrogaster",
    genotype=metatable.Genotype[i],
)

# ADD METADATA ABOUT EXPERIMENT
metadata["NWBFile"].update(
    session_id=f"{metatable.AssayType[i]}",
    session_description=f"Fiber photometry recording of a prairie vole interacting in an assay of type: {metatable.AssayType[i]}.",
    experiment_description="Data associated with publication: Loss of oxytocin receptor function disrupts neural signatures of pair bonding and fidelity in the nucleus accumbens.",
    experimenter=["Long, Kimberly L. P.",],
    lab="Devanand Manoli",
    institution="University of California, San Francisco",
)

# ADD PHOTOMETRY METADATA
metadata = dict_deep_update(metadata, fiber_photometry_metadata)           

# RUN CONVERSION WITH IN-MEMORY FILE
nwbfile = TDTFiberPhotometryInterface.create_nwbfile(interface,metadata=metadata)

# ADD LAB SPECIFIC METADATA
atype = metatable.AssayType[i]
if math.isnan(metatable.PartnerEartag[i]):
    pID = 'unknown'
else:
    pID = metatable.PartnerEartag[i]
pGT = 'WT'
if math.isnan(metatable.PartnerSex[i]):
    if metatable.Sex[i]=='F':
        pSex='M'
    else:
        pSex='F'
else:
    pSex = metatable.PartnerSex[i]
if math.isnan(metatable.PartnerFamily[i]):
    pFam = 'unknown'
else:
    pFam = metatable.PartnerFamily[i]
if math.isnan(metatable.PartnerDOB[i]):
    pDOB = 'unknown'
else:
    pDOB = str(metatable.PartnerDOB[i])
labmeta = make_metadata_object(atype,pID,pGT,pSex,pFam,pDOB,dpostpair)
nwbfile.add_lab_meta_data(lab_meta_data=labmeta)

# ADD VIDEO INFORMATION
# !!! write a utility to copy all of the video files from tanks to another directory for uploading
path_to_bulk_videos = os.path.join('..','behavior_videos')
# get video file path from TDT tank
vids = []
vids += [each for each in os.listdir(folder_path) if each.endswith('.avi')]
path_to_vid = os.path.join(path_to_bulk_videos,vids[0])
# get video timestamps from TDT
tstampstruct = tdt.read_block(folder_path,evtype=['epocs'])
tstamps = tstampstruct.epocs.Cam1.onset
video_ext_file = ImageSeries(
    name='behavior_video',
    description='Raw original video collected by TDT RZ5P during photometry recording and timestamped synchronously to fluorescence recording.',
    unit='n.a.',
    external_file=[path_to_vid],
    format='external',
    timestamps=tstamps,
)
# add to NWB file
nwbfile.add_acquisition(video_ext_file)

# ADD SCORING TABLES
fullscore = os.path.join(metatable.ScorePath[i],metatable.ScoreFile[i])
anno = pd.read_csv(fullscore,header=15,usecols=['Time','Behavior','Status'])
# need to reorganize so events are listed with start_time and stop_time
BORIS_annotations = TimeIntervals(
    name="raw_manual_behavior_annotations",
    description="Full description of animal behavior during video, manually annotated in BORIS software. Each behavior is timestamped at the START and STOP (or a single frame for POINT events).",
)
BORIS_annotations.from_dataframe(anno)

# ADD PREPROCESSED DATA

# WRITE FULLY FILLED OUT FILE
# configure_and_write_nwbfile(
#     nwbfile, nwbfile_path="path/to/destination.nwb", backend="hdf5"
# )

read from t=0s to t=3543.61s


In [30]:
BORIS_annotations = TimeIntervals(
    name="raw_manual_behavior_annotations",
    description="Full description of animal behavior during video, manually annotated in BORIS software. Each behavior is timestamped at the START and STOP (or a single frame for POINT events).",
)
BORIS_annotations.from_dataframe(anno,"raw_manual_behavior_annotations")

ValueError: missing required cols: {'start_time', 'stop_time'}

In [13]:
# STATIC FIBER PHOTOMETRY METADATA

fiber_photometry_metadata = {
    "Ophys": {
        "FiberPhotometry": {
            "OpticalFibers": [
                {
                    "name": "optical_fiber",
                    "model": "MFC_400/430-0.48_6.5mm_MF1.25_FLT",
                    "numerical_aperture": 0.48,
                    "core_diameter_in_um": 400.0
                }
            ],
            "ExcitationSources": [
                {
                    "name": "excitation_source_calcium_signal",
                    "model": "Doric_Lenses_Connectorized_Single_LED_Blue_465",
                    "illumination_type": 'LED',
                    "excitation_wavelength_in_nm": 465.0
                },
                {
                    "name": "excitation_source_red_control_signal",
                    "model": "Doric_Lenses_Connectorized_Single_LED_Lime_560",
                    "illumination_type": 'LED',
                    "excitation_wavelength_in_nm": 560.0
                }
            ],
            "Photodetectors": [
                {
                    "name": "photodetector_green",
                    "model": "NPM_2151",
                    "detector_type": 'photodiode',
                    "detected_wavelength_in_nm": 520.0
                },
                {
                    "name": "photodetector_red",
                    "model": "NPM_2151",
                    "detector_type": 'photodiode',
                    "detected_wavelength_in_nm": 630.0
                }
            ],
            "DichroicMirrors": [
                {
                    "name": "dichroic_mirror",
                    "model": "FMC5_E1(460-490)_F1(500-540)_E2(550-580)_F2(600-680)_S"
                }
            ],
            "BandOpticalFilter":[
                {
                    "name":  "5 port Fluorescence Mini Cube (F1)",
                    "center_wavelength_in_nm": 520.0,
                    "bandwidth_in_nm": 40.0,
                    "filter_type": "bandpass",
                    "model": "FMC5_E1(460-490)_F1(500-540)_E2(550-580)_F2(600-680)_S"
                },
                {
                    "name":  "5 port Fluorescence Mini Cube (F2)",
                    "center_wavelength_in_nm": 640.0,
                    "bandwidth_in_nm": 80.0,
                    "filter_type": "bandpass",
                    "model": "FMC5_E1(460-490)_F1(500-540)_E2(550-580)_F2(600-680)_S"
                },  
            ],
            "Indicators": [
                {
                    "name": "NAcc_GCaMP_injection",
                    "description": "AAV8-Syn-GCaMP6m-WPRE viral vector used to record Ca2+ fluctuations (1.00e+13, Vigene).",
                    "label": "GCaMP6m",
                    "injection_location": "NAcc shell",
                    "injection_coordinates_in_mm": [1.7,1.2,-5.05],
                },
                {
                    "name": "NAcc_tdTomato_injection",
                    "description": "AAV8-CAG-tdTomato viral vector used as a static control (5.00e+12, Dr. Ed Boyden lab via UNC Vector Core).",
                    "label": "tdTomato",
                    "injection_location": "NAcc shell",
                    "injection_coordinates_in_mm": [1.7,1.2,-5.05],
                }
            ],
            "FiberPhotometryTable": {
                "name": "fiber_photometry_table",
                "description": "Fiber photometry system metadata table.",
                "rows": [
                    {
                        "name": "0",
                        "location": "NAcc",
                        "excitation_wavelength_in_nm": 465.0,
                        "emission_wavelength_in_nm": 525.0,
                        "indicator": "NAcc_GCaMP_injection",
                        "optical_fiber": "optical_fiber",
                        "excitation_source": "excitation_source_calcium_signal",
                        "photodetector": "photodetector_green",
                        "dichroic_mirror": "dichroic_mirror"
                    }
                ]
            },
            "FiberPhotometryResponseSeries": [
                {
                    "name": "all_TDT_data",
                    "description": "Photometry recordings were conducted with TDT RZ5P acquisition system and Synapse software. Data is 4xn with row 0 corresponding to voltage data from GCaMP excitation LED, row 1 -> tdTomato LED, row 2 -> GCaMP response data, row 3 -> tdTomato response data.",
                    "stream_name": "Fi1r",
                    "unit": "a.u.",
                    "fiber_photometry_table_region": [0],
                    "fiber_photometry_table_region_description": "The region of the FiberPhotometryTable corresponding to the calcium signal."
                }
            ]
        }
    }
}

In [14]:
def make_metadata_object(atype,pID,pGT,pSex,pFam,pDOB,dpostpair,sID=None,sSex=None,sFam=None,sDOB=None,chamber=None):
        
    match atype:
        case 'introduction':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=1800.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Implanted animal is introduced to a novel partner in a clean cage.',
                assay_type__days_post_pairing=dpostpair,
            )
            
        case 'timed_mating':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=1800.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Implanted animal and cohabitating partner are allowed free access after ~24 hours separated by a barrier.',
                assay_type__days_post_pairing=dpostpair,
            )
                      
           
        case 'PPT':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=10800.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Implanted animal is allowed to freely move through a 3-chamber apparatus with cohabitating partner tethered in one chamber and a novel opposite-sex stranger tethered in opposite chamber.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__stranger_ID=sID,
                assay_type__stranger_GT='WT',
                assay_type__stranger_sex=sSex,
                assay_type__stranger_fam=sFam,
                assay_type__stranger_DOB=sDOB,
                assay_type__PPT_lane=1,
                assay_type__partner_chamber=chamber,
            )

        case 'separation_reun':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=1800.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Focal animal remains in home cage while partner is removed to a clean cage for 60 mins. Partner is then returned to the cage with the focal animal.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__isolation_length=3600.0,
            )

        case 'aggression':
            metaObj = AssayMetadata(
                assay_type=atype,
                duration=1200.0,
                exclude_flag=False,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__partner_fam=pFam,
                assay_type__partner_DOB=pDOB,
                assay_type__description='Focal animal remains in home cage while partner is removed to a clean cage for 60 mins. A novel opposite-sex stranger is then introduced to the cage with the focal animal.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__stranger_ID=sID,
                assay_type__stranger_GT='WT',
                assay_type__stranger_sex=sSex,
                assay_type__stranger_fam=sFam,
                assay_type__stranger_DOB=sDOB,
                assay_type__isolation_length=3600.0,
            )
            
    return metaObj

In [8]:
math.isnan(metatable.PartnerSex[0])

True

In [4]:
folder_path

'M:\\photometry-Long2025\\tdt_files\\1 Intros\\V1259-181205-134533'